In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import ssm
import matplotlib.pyplot as plt
import plotly.express as px
import pickle
import scipy
import scipy.io
import tsfresh
import dtale
import os
px.set_mapbox_access_token("pk.eyJ1IjoibWlrb2xhanNsdXBpbnNraSIsImEiOiJjazJ5dDI0MWEwOTA3M2hxanRwbmo4NDRuIn0.NSbj3B1Bm6fC5QUP3s-BjQ")
import plotly.io as pio
pio.renderers.default = "iframe"
import dask
from lib.huawei import load_data, load_motion, load_ambient, load_battery, load_api, load_label
import tables
from ssm.util import random_rotation, find_permutation
import logging
logging.basicConfig(filename='huawei_slds.log', level=logging.DEBUG)

In [3]:
with open("../Data/huawei-competition/selected_features.pickle", "rb") as ifile:
    selected_features = pickle.load(ifile)

In [4]:
data = []
for selected_feature in selected_features:
#     file_path = os.path.join("../Data/2012.01.17-huawei/splitted/", f"1.3a-stationary_50ms__{selected_feature}.csv")
#     df = pd.read_csv(file_path, parse_dates = ["DATETIME_UTC"], index_col="DATETIME_UTC")
    a = np.load(os.path.join("../", selected_feature))
    data.append(a)
#     print(df.shape)
data = np.stack(data, axis=1)

In [5]:
labels = np.load("../Data/huawei-competition/resampled/Label.npy")[:,-1].astype(int) - 1

In [6]:
order = np.loadtxt("../Data/huawei-competition/train_order.txt").astype(int)

In [7]:
data_ordered = np.zeros(data.shape)

In [8]:
data_ordered[order - 1, :] = data

In [9]:
labels_ordered = np.zeros(labels.shape)
labels_ordered[order - 1] = labels

In [19]:
def train_model(X_train,
                y_train,
                i, j,
                transitions="recurrent",
                dynamics="diagonal_gaussian",
                emissions="gaussian",
                single_subspace=False,
                num_iters=10000,
                num_init_iters=10000,
                num_init_restarts=3,
                discrete_state_init_method = "kmeans"
               ):
    arhmm = ssm.hmm.HMM(2, 100,
                            transitions=transitions,
                            observations=dynamics)
    hmm_lls = arhmm.fit(X_train, num_iters=num_iters,
             num_init_iters=num_init_iters, init_method = discrete_state_init_method)
    most_likely = arhmm.most_likely_states(X_train)
    try:
        perm = find_permutation(y_train.astype(int), most_likely)
        arhmm.permute(perm)
    except:
        logging.error("Permutation exception")
    with open(f"../Models/binary_huawei_rarhmm_{transitions}_{dynamics}_{num_iters}_{num_init_iters}_{discrete_state_init_method}_labels_{i}_{j}.pickle", "wb") as ofile:
        pickle.dump({
            "arhmm": arhmm,
            "hmm_lls": hmm_lls,
            "most_likely": most_likely,
        }, ofile)

In [26]:
params_list = [
    {
        "transitions": "recurrent",
        "dynamics": "diagonal_gaussian",
        "num_iters": 10000,
        "discrete_state_init_method": "random"
    },
    {
        "transitions": "recurrent",
        "dynamics": "diagonal_gaussian",
        "num_iters": 10000,
        "discrete_state_init_method": "kmeans"
    },
]

In [27]:
done = []

In [ ]:
for i in range(8):
    for j in range(i):
        if (i, j) in done:
            continue
        indices = np.isin(labels_ordered.astype(int), [i, j])
        b_data_ordered = data_ordered[indices]
        b_labels_ordered = labels_ordered[indices]
        b_labels_ordered[b_labels_ordered.astype(int) == i] = -1
        b_labels_ordered[b_labels_ordered.astype(int) == j] = -2
        b_labels_ordered = b_labels_ordered + 1
        n = int(b_data_ordered.shape[0]*0.25)
        m = int(b_data_ordered.shape[0]*0.5)
        X_train = b_data_ordered[m:]
        y_train = b_labels_ordered[m:]
        X_val = b_data_ordered[:n]
        y_val = b_labels_ordered[:n]
        X_test = b_data_ordered[n:m]
        y_test = b_labels_ordered[n:m]
        for params in params_list:
            try:
                train_model(X_train, y_train, i, j, **params)
            except Exception as e:
                logging.error(f"{e}")

  0%|          | 0/10000 [00:01<?, ?it/s]

In [ ]:
posterior_x = posterior.mean_continuous_states[0]
most_likely = rslds.most_likely_states(posterior_x, X_train)

In [ ]:
y_train

In [ ]:
# try:
perm = find_permutation(y_train.astype(int), most_likely)
rslds.permute(perm)
# except:
#     logging.error("Prediction exception")

In [ ]:
def plot_states(data_z, z_est, label, gen):
    titles = ["True", label]
    states_list = [data_z, z_est]
    fig, axs = plt.subplots(2,1, figsize=(6,4))
    for (i, ax, states) in zip(range(len(axs)), axs, states_list):
        ax.imshow(states[None,:], aspect="auto")
        ax.set_yticks([])
        ax.set_title(titles[i])
        if i < (len(axs) - 1):
            ax.set_xticks([])

    plt.suptitle(f"{titles[0]} and {titles[1]} States: {gen}", va="baseline")
    plt.tight_layout()
    plt.savefig(f"../Plots/{titles[0]}_{titles[1]}_{gen}.pdf")
    plt.show()

In [ ]:
z_est = rslds.most_likely_states(posterior_x, X_train)
plot_states(y_train, z_est, "Predicted", 0)

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix

In [ ]:
accuracy_score(y_train, z_est)

In [ ]:
confusion_matrix(y_train, z_est)